In [9]:
example = """
Player 1:
9
2
6
3
1

Player 2:
5
8
4
7
10
"""[1:-1].splitlines()

with open("day22.txt", "r") as f:
    data = f.readlines()

In [12]:
from typing import List, Tuple

def parse_input(lines: List[str]) -> (Tuple[int], Tuple[int]):
    player_decks = ([], [])
    current_player = 0

    for line in lines:
        line = line.strip()
        if line.startswith("Player "):
            current_player = int(line[7:-1]) - 1
        elif line.isdigit():
            player_decks[current_player].append(int(line))

    return tuple(tuple(deck) for deck in player_decks)

assert parse_input(example) == (
    (9, 2, 6, 3, 1),
    (5, 8, 4, 7, 10)
)

In [38]:
from typing import Iterable, Tuple

class Combat(object):
    def __init__(self, player1: Iterable[int], player2: Iterable[int]):
        self.player1 = list(player1)
        self.player2 = list(player2)

    def run(self) -> Tuple[int, int]:
        winner = None
        while winner is None:
            winner = self.step()

        return (winner, self.score()[winner - 1])

    def step(self) -> int:
        game_winner = self.get_game_winner()

        if game_winner is not None:
            return game_winner

        player1 = self.player1.pop(0)
        player2 = self.player2.pop(0)

        round_winner = self.get_round_winner(player1, player2)
        if round_winner == 1:
            self.player1.extend([player1, player2])
        else:
            self.player2.extend([player2, player1])

        return None

    def get_game_winner(self) -> int:
        if not self.player1:
            return 2

        if not self.player2:
            return 1

        return None

    def get_round_winner(self, player1: int, player2: int) -> int:
        if player1 > player2:
            return 1
        else:
            return 2

    def score(self) -> Tuple[int, int]:
        return (
            sum(card * (i + 1) for i, card in enumerate(reversed(self.player1))),
            sum(card * (i + 1) for i, card in enumerate(reversed(self.player2)))
        )

example_game = Combat(*parse_input(example))
assert example_game.run() == (2, 306)

true_game = Combat(*parse_input(data))
print(f"Final Winner and Score (part 1): {true_game.run()}")

Final Winner and Score (part 1): (1, 31957)


In [40]:
from typing import Iterable, Set, Tuple

class RecursiveCombat(Combat):
    def __init__(self, player1: Iterable[int], player2: Iterable[int], prior_states: Set[Tuple[Tuple[int], Tuple[int]]] = None):
        self.player1 = list(player1)
        self.player2 = list(player2)

        self.prior_states = prior_states or set()
    
    def state_snapshot(self):
        return (tuple(self.player1), tuple(self.player2),)

    def get_game_winner(self) -> int:
        state = self.state_snapshot()
        if state in self.prior_states:
            return 1

        self.prior_states.add(state)

        return super().get_game_winner()

    def get_round_winner(self, player1: int, player2: int) -> int:
        if player1 <= len(self.player1) and player2 <= len(self.player2):
            sub_game = RecursiveCombat(self.player1[:player1], self.player2[:player2])
            winner, score = sub_game.run()
            return winner
        
        return super().get_round_winner(player1, player2)
        
example_game = RecursiveCombat(*parse_input(example))
example_result = example_game.run()
print(f"Example Recursive Result (part 2): {example_result}")
assert example_result == (2, 291)

true_game = RecursiveCombat(*parse_input(data))
true_result = true_game.run()
print(f"Recursive Result (part 2): {true_result}")


Example Recursive Result (part 2): (2, 291)
Recursive Result (part 2): (1, 33212)
